In [1]:
print("hello world") # testing

hello world


In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv");

In [4]:
print(df.head(5))

  artist                   song                                        link  \
0   ABBA  Ahe's My Kind Of Girl  /a/abba/ahes+my+kind+of+girl_20598417.html   
1   ABBA       Andante, Andante       /a/abba/andante+andante_20002708.html   
2   ABBA         As Good As New        /a/abba/as+good+as+new_20003033.html   
3   ABBA                   Bang                  /a/abba/bang_20598415.html   
4   ABBA       Bang-A-Boomerang      /a/abba/bang+a+boomerang_20002668.html   

                                                text  
0  Look at her face, it's a wonderful face  \r\nA...  
1  Take it easy with me, please  \r\nTouch me gen...  
2  I'll never know why I had to go  \r\nWhy I had...  
3  Making somebody happy is a question of give an...  
4  Making somebody happy is a question of give an...  


In [5]:
print(df.tail(5))

             artist           song  \
57645  Ziggy Marley  Good Old Days   
57646  Ziggy Marley  Hand To Mouth   
57647          Zwan   Come With Me   
57648          Zwan         Desire   
57649          Zwan      Heartsong   

                                              link  \
57645  /z/ziggy+marley/good+old+days_10198588.html   
57646  /z/ziggy+marley/hand+to+mouth_20531167.html   
57647           /z/zwan/come+with+me_20148981.html   
57648                 /z/zwan/desire_20148986.html   
57649              /z/zwan/heartsong_20148991.html   

                                                    text  
57645  Irie days come on play  \r\nLet the angels fly...  
57646  Power to the workers  \r\nMore power  \r\nPowe...  
57647  all you need  \r\nis something i'll believe  \...  
57648  northern star  \r\nam i frightened  \r\nwhere ...  
57649  come in  \r\nmake yourself at home  \r\ni'm a ...  


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [9]:
df.head(10)

,artist,song,text
0,Kiss,King Of The Night Time World,"It's so sad, livin' at home \r\nFar from the ..."
1,Def Leppard,C'mon C'mon,"Everybody, everyone c'mon c'mon \r\nC'mon c'm..."
2,Robbie Williams,Angel,I sit and wait \r\nDoes an angel contemplate ...
3,Jimmy Buffett,Creola,Creola in my soul-a \r\nI loved what they wer...
4,Regine Velasquez,Bluer Than Blue,"After you go, I can catch on my reading \r\nA..."
5,George Jones,Don't Stop The Music,Don't stop the music let it play one more song...
6,Ed Sheeran,Autumn Leaves,Another day another life \r\nPasses by just l...
7,Gino Vannelli,My Oh My It's A Miracle,My oh my it's a miracle \r\nGood morning love...
8,Waylon Jennings,Baker Street,Winding your way down on Baker Street \r\nLit...
9,Hank Snow,Broken Hearted,"You ask me why I'm broken hearted \r\nWhy, I'..."


In [10]:
df['text'][0]

"It's so sad, livin' at home  \r\nFar from the city and the midnight fun  \r\nIt's so bad, goin' to school  \r\nSo far from me and the dirty things that we do  \r\nI'm the king of the night time world  \r\nAnd you're my headlight queen  \r\nI'm the king of the night time world  \r\nCome live your secret dream  \r\nIt's so fine, lovin' with ease  \r\nFar from the house and the family fights  \r\nIt's so fine, bein' with you  \r\nBein' with me makes everything alright  \r\nI'm the king of the night time world  \r\nAnd you're my headlight queen  \r\nI'm the king of the night time world  \r\nCome live your secret dream, alright  \r\nIt's so sad, you're not content  \r\nFar from the music and the neon glow  \r\nAin't you glad we got the time  \r\nFar from our folks, they'll never ever know  \r\nI'm the king of the night time world  \r\nAnd you're my headlight queen  \r\nI'm the king of the night time world  \r\nCome live your secret dream  \r\nI'm the king, I'm the king, I'm the king, I'm t

In [11]:
df.shape

(5000, 3)

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s','').replace(r'\n',' ',regex = True)

In [21]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [22]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [23]:
!pip install scikit-learn

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [27]:
similarity[0]

array([1.        , 0.00706159, 0.00955081, ..., 0.00163735, 0.03327861,
       0.0432749 ])

In [28]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [29]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [31]:
recommendation('Bluer Than Blue')

["Hope She'll Be Happier",
 'Blue Train',
 "I'm Looking For Blue Eyes",
 'Blue',
 'How Much I Lied',
 'Midnight Blue',
 'Blue',
 "I'd Rather Be Blue Over You (Than Happy With Somebody Else)",
 'Gonna Love Ya',
 'Blue Island',
 'Baby Blue',
 'Blue Turns To Grey',
 'In My Life',
 'Singing The Blues',
 'Black And Blue',
 'Jet Boy Blue',
 'High Lonesome Sound',
 'Bye Bye Baby Blues',
 'Till Your Well Runs Dry',
 'Turn']

In [33]:
import pickle

In [34]:
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))